Find Approximate Nearest Neighbors

Dataset used- Movielens 

In [ ]:
!apt install libomp-dev
!pip install faiss
!pip install nmslib
!pip install annoy

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libomp-dev is already the newest version (5.0.1-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
     |████████████████████████████████| 646 kB 3.9 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391672 sha256=5a4a593caf7e49ab5b69b06f0ea9ca6dc1a1e95a01d9296bb3c083e01f024e9e
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


Import the neccessary modules

In [ ]:
import pickle
import faiss
import nmslib
import annoy

Load the data

In [ ]:
def load_data():
    with open('movies.pickle', 'rb') as f:
        data = pickle.load(f)
    return data

data = load_data()
vectors = data["vector"]
names = data["name"]
data

{'name': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'vector': array([[-0.01780608, -0.14265831,  0.10308606, ...,  0.09659795,
         -0.17529577, -0.03061521],
        [-0.03357764,  0.16418771,  0.21801303, ...,  0.16502103,
         -0.09166156,  0.05047869],
        [-0.2761452 , -0.01991325, -0.04969981, ...,  0.0258275 ,
         -0.08328608, -0.0152858 ],
        ...,
        [ 0.05142734, -0.01683608, -0.20441587, ...,  0.00045828,
          0.14679626,  0.2462584 ],
        [ 0.04491899, -0.02819411, -0.09472758, ..., -0.02152078,
          0.16223577,  0.19897607],
        [ 0.02531924,  0.03099714,  0.06437534, ..., -0.07260127,
          0.0467432 ,  0.07893164]], dtype=float32)}

In [ ]:
faiss.MatrixStats(vectors).comments.split("\n")

['analyzing 1682 vectors of size 64',
 'no NaN or Infs in data',
 'all vectors are distinct',
 'range of L2 norms=[0.747558, 1.80436] (0 null vectors)',
 'matrix contains no 0s',
 'no constant dimensions',
 'no dimension has a too large mean',
 'stddevs per dimension are in [0.112036 0.158214]',
 '']

**LHS**

In [ ]:
class LSHIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self, num_bits=8):
        self.index = faiss.IndexLSH(self.dimension, num_bits)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

In [ ]:
index = LSHIndex(data["vector"], data["name"])
index.build()

In [ ]:
movie_vector, movie_name = data['vector'][90:91], data['name'][90]
simlar_movies_names = '\n* '.join(index.query(movie_vector))
print(f"The most similar movies to {movie_name} are:\n* {simlar_movies_names}")

The most similar movies to Nightmare Before Christmas, The (1993) are:
* Secrets & Lies (1996)
* Nightmare Before Christmas, The (1993)
* Welcome to the Dollhouse (1995)
* Twelve Monkeys (1995)
* Richard III (1995)
* Madness of King George, The (1994)
* Horseman on the Roof, The (Hussard sur le toit, Le) (1995)
* Crumb (1994)
* Angels and Insects (1995)
* Brothers McMullen, The (1995)


Exhaustive Search by Faiss

In [ ]:
index = faiss.IndexFlatL2(vectors.shape[1])
index.add(vectors)

In [ ]:
search_vector = vectors[90:91]
distances, indices = index.search(search_vector, 10)

In [ ]:
print(f"The most imilar movies to {names[91]} are:\n")
print([names[i] for i in indices[0]])

The most similar movies to True Romance (1993) are:

['Nightmare Before Christmas, The (1993)', 'Heavy Metal (1981)', 'Sirens (1994)', 'Beauty and the Beast (1991)', 'Akira (1988)', 'Fantasia (1940)', 'Benny & Joon (1993)', 'Barbarella (1968)', "Pete's Dragon (1977)", 'James and the Giant Peach (1996)']


Vector Encoding 

In [ ]:
quantizer = faiss.IndexFlatL2(vectors.shape[1])
index = faiss.IndexIVFPQ(quantizer, 
                         vectors.shape[1], 
                         100,             # number_of_partition,
                         8,               # search_in_x_partitions, 
                         8)               # subvector_size
index.train(vectors)
index.add(vectors)

In [ ]:
search_vector = vectors[90:91]
distances, indices = index.search(search_vector, 10)

In [ ]:
print(f"The most similar movies to {names[90]} are:\n")
print([names[i] for i in indices[0]])

The most similar movies to Nightmare Before Christmas, The (1993) are:

['Nightmare Before Christmas, The (1993)', 'Heavy Metal (1981)', 'Pink Floyd - The Wall (1982)', 'Akira (1988)', 'Hamlet (1996)', 'Full Metal Jacket (1987)', 'Supercop (1992)', 'Scream of Stone (Schrei aus Stein) (1991)', 'Scream of Stone (Schrei aus Stein) (1991)', 'Scream of Stone (Schrei aus Stein) (1991)']


**Annoy**

In [ ]:
class AnnoyIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimention)
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(vector.tolist(), k)
        return [self.labels[i] for i in indices]

In [ ]:
index = AnnoyIndex(data["vector"], data["name"])
index.build()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  if __name__ == '__main__':


In [ ]:
movie_vector, movie_name = data['vector'][90], data['name'][90]
simlar_movies_names = '\n* '.join(index.query(movie_vector))
print(f"The most similar movies to {movie_name} are:\n* {simlar_movies_names}")

The most similar movies to Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
* Aladdin (1992)
* Blade Runner (1982)
* Aliens (1986)
* Pink Floyd - The Wall (1982)
* Brazil (1985)
* Wrong Trousers, The (1993)
* Alien (1979)
* Return of the Jedi (1983)
* E.T. the Extra-Terrestrial (1982)


HNSW

In [ ]:
class NMSLIBIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels

    def build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({'post': 2})
        
    def query(self, vector, k=10):
        indices = self.index.knnQuery(vector, k=k)
        return [self.labels[i] for i in indices[0]]

In [ ]:
index = NMSLIBIndex(data["vector"], data["name"])
index.build()

In [ ]:
movie_vector, movie_name = data['vector'][90], data['name'][90]
simlar_movies_names = '\n* '.join(index.query(movie_vector))
print(f"The most similar movies to {movie_name} are:\n* {simlar_movies_names}")